In [18]:
import os
import json

# import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization, Input, Concatenate, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l1

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
from imutils import paths
from PIL import Image
from tqdm import tqdm
import shutil
import tarfile
from multiprocessing import Manager, Process, Pool

In [19]:
target_size = (224,224)
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = sum([len(files) for r,d,files in os.walk('train')])
nb_validation_samples = sum([len(files) for r,d,files in os.walk('validation')])
epochs = 50
batch_size = 32

In [20]:
with open('train_annotations.json', 'r') as file:
    train_annotations = json.load(file)

with open('validation_annotations.json', 'r') as file:
    validation_annotations = json.load(file)

In [ ]:
train_data = Manager().list()
train_labels = Manager().list()
    
def train_label_worker(i, train_annotations, length):
    for fc in range(i,i+int(length/4)):
        if fc>=length: return
        file_name = list(paths.list_images('train'))[fc]
        image = cv2.imread(file_name)
        image = cv2.resize(image, (target_size[1], target_size[0]))
        train_data.append(img_to_array(image))
        labels = set()
        for category in train_annotations[file_name.split(os.path.sep)[-1]]['categories']: labels.add(category)
        train_labels.append(list(labels))

processPool = Pool(4)
processPool.starmap(train_label_worker, [(i,train_annotations,len(os.listdir('train'))) for i in [0,4135,8270,12405]])

print(train_labels)

In [1]:
validation_data = Manager().list()
validation_labels = Manager().list()

def validation_label_worker(i, validation_annotations, length):
    for fc in range(i,i+int(length/4)):
        if fc>=length: return
        file_name = list(paths.list_images('validation'))[fc]
        image = cv2.imread(file_name)
        image = cv2.resize(image, (target_size[1], target_size[0]))
        train_data.append(img_to_array(image))
        labels = set()
        for category in validation_annotations[file_name.split(os.path.sep)[-1]]['categories']: labels.add(category)
        validation_labels.append(list(labels))

processPool = Pool(4)
processPool.starmap(train_label_worker, [(i,train_annotations,len(os.listdir('train'))) for i in [0,12417,24834,37251]])

print(validation_labels)

NameError: name 'Manager' is not defined